In [1]:
import scipy as sp

In [8]:
%run '/home/ihs01/Documents/NOTEBOOKS/import_getters.ipynb'

## Statistical Analysis

In [9]:
def getStats(sensor, eventTupleList):
    ''' NAME
            getStats

        DESCRIPTION
            This function uses a list of eventTuples or single tuple and performs statistical analysis 
            on sensor data for each tuple to return a dataframe with mean, max, min, std, skew, kurtosis, 
            and tuple info.  

        INPUT
            sensor:           name of sensor to get data for
            eventTupleList:   can be either a list of eventTuples or a single tuple

        RETURN
            returns a dataframe containing statistical calculations for each event with an index of event ID, 
            reactor, and run

        EXAMPLE
            1.) Multiple Events
            #Get list of Argon events
            dataBase ='rct007'
            gasType ='Argon'
            gasEventsDf = getGasEvents(dataBase,gasType)

            #Get list of tuples for Argon events
            eventTupleList = gasEventsDf.index
            sensor = 'rct_rtd1'

            getStats(sensor, eventTupleList)

            2.) Single Event
            eventTuple = getEvents('rct007').index
            eventNumber = 3
            singleEvent = [eventTuple[eventNumber]]
            sensor = 'rct_rtd1'

            getStats(sensor, singleEvent)'''
    
    list1 = ['mean', 'max', 'min', 'std', 'skew', 'kurt']
    stats = pd.DataFrame(columns = list1)
    df = pd.DataFrame()
    for eventTup in eventTupleList:

        #Get data
        [datetime, data] = getEngData(sensor, eventTup)
        
        #Analyze data
        mean = data.mean()
        maxval = data.max()
        minval = data.min()
        std = data.std()   
        skew1 = sp.stats.skew(data)
        skew = skew1[0]
        kurtosis1 = sp.stats.kurtosis(data, fisher=False)
        kurtosis = kurtosis1[0]
        
        #Compile data
        A = [[mean], [maxval], [minval], [std], [skew], [kurtosis]]
        B = zip(*A)
        df1 = pd.DataFrame(B, columns = list1)        
        stats = pd.concat([stats, df1], axis=0)
    
    #Re-index dataframe with eventTuple info
    index = pd.MultiIndex.from_tuples(eventTupleList, names=['event id', 'reactor', 'run'])
    stats = stats.set_index(index)
    
    return stats

## Calculate Input Power

In [ ]:
def findInputPower(eventTuple):
    
    ''' NAME
            findInputPower
        
        DESCRIPTION
            This function calculates the input power of the fan and reactor heaters
        
        INPUT
            eventTuple:  contains event ID, run, and database info
        
        RETURN
            returns a calculated value for input power
        
        EXAMPLE
            eventTuple = getEvents('rct007').index
            
            findInputPower(eventTuple[12])'''
    
    lineVoltageSensor = 'v5'
    fanCurrentSensor = 'v8'
    htr1CurrentSensor = 'v1'
    htr2CurrentSensor = 'v4'
    
    datetime, lineVoltage = getEngData(lineVoltageSensor, eventTuple)
    datetime, fanCurrent = getEngData(fanCurrentSensor, eventTuple)
    datetime, h1Current = getEngData(htr1CurrentSensor, eventTuple)
    datetime, h2Current = getEngData(htr2CurrentSensor, eventTuple)

    pwrFan =  lineVoltage * fanCurrent
    pwrHtr1 = lineVoltage * h1Current
    pwrHtr2 = lineVoltage * h2Current
    
    meanPwrFan = pwrFan.mean()
    stdPwrFan = pwrFan.std()
    meanPwrHtr1 = pwrHtr1.mean()
    stdPwrHtr1 = pwrHtr1.std()
    meanPwrHtr2 = pwrHtr2.mean()
    stdPwrHtr2 = pwrHtr2.std()

    return meanPwrFan+meanPwrHtr1+meanPwrHtr2

## Power Out

In [ ]:
def density(temp):
    Density = (1.626*(10**-5)*temp**3-6.021*(10**-3)*temp**2+0.021*temp + 1000.066)/1000
    return Density

def specificHeat(temp):
    SH = -3.588*(10**-11)*(temp**5)+1.212*(10**-8)*(temp**4)-1.564*(10**-6)*(temp**3)+1.036*(10**-4)*(temp**2)-3.266*(10**-3)*temp + 4.216
    return SH

def outputPwr(eventTuples, path = '/home/ihs01/Documents/NOTEBOOKS/DATA/'):
    #read in flow data
    fi = getEngData('flowin',eventTuples, path = '/home/ihs01/Documents/NOTEBOOKS/DATA/')
    fo = getEngData('flowout',eventTuples, path = '/home/ihs01/Documents/NOTEBOOKS/DATA/')
    fit = getEngData('flowin_rtd',eventTuples,path = '/home/ihs01/Documents/NOTEBOOKS/DATA/')
    fot = getEngData('flowout_rtd',eventTuples,path = '/home/ihs01/Documents/NOTEBOOKS/DATA/')
    
    #temperatures
    fitmean = fit[1].mean()
    fotmean = fot[1].mean()
    deltaT = fotmean-fitmean

    #total volume in event
    volumeInDelta = fi[1][-1] - fi[1][0]

    #duration of event
    dti = (fi[0][-1]-fi[0][0])
    dtsi = dti.seconds
    
    #standard deviation of flow in
    dateSDi = np.diff(fi[0]-fi[0][0])/np.timedelta64(1, 's')
    fistd = np.std((fi[1][1:]-fi[1][:-1])/dateSDi)
    fistd
    
    #Q flow in
    Qin = volumeInDelta/dtsi*density(fitmean)*specificHeat(fitmean)*deltaT*1000
    
    #total volume out event
    volumeOutDelta = fo[1][-1] - fo[1][0]
    
    #duration of event
    dto = (fo[0][-1] - fo[0][0])
    dtso = dto.seconds
    
    #standard deviation of flow out
    dateSDo = np.diff(fo[0]-fo[0][0])/np.timedelta64(1, 's')
    fostd = np.std((fo[1][1:]-fo[1][:-1])/dateSDo)
    
    #Q flow out
    Qout = volumeOutDelta/dtso*density(fotmean)*specificHeat(fotmean)*deltaT*1000

    return {'dT:':deltaT, 'IN Total Volume:':volumeInDelta, 'IN Duration:':dtsi, 'Q in:':Qin, 'OUT Total Volume:':volumeOutDelta, 'OUT Duration:':dtso, 'Q out:':Qout}
